In [1]:
from bs4 import BeautifulSoup
import logging
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
link_template = 'https://dblp.org/db/conf/kdd/kdd2024.html'
def get_title_author(link):
    resp = requests.get(link, proxies=None)
    soup = BeautifulSoup(resp.content, 'html.parser')
    uls = soup.find_all('ul', class_='publ-list')
    titles, authors = [], []
    for ul in uls[2:7]:  # 遍历所有 ul
        articles = ul.find_all('li', class_='entry inproceedings')  # 找到当前 ul 下的所有 li
        print(f"Found {len(articles)} articles in one ul.")
        for li in articles:
            title = li.find('span', class_='title').text
            author_list = [a.text for a in li.find_all('span', itemprop='name')]

            titles.append(title)
            authors.append(", ".join(author_list[:-1]))

    return titles, authors

In [3]:
titles, authors = get_title_author(link_template)
print(titles, authors)

Found 411 articles in one ul.
Found 5 articles in one ul.
Found 151 articles in one ul.
Found 9 articles in one ul.
Found 25 articles in one ul.
['GEO: Generative Engine Optimization.', 'Approximating Memorization Using Loss Surface Geometry for Dataset Pruning and Summarization.', 'Resilient k-Clustering.', 'Statistical Models of Top-k Partial Orders.', 'A Learned Generalized Geodesic Distance Function-Based Approach for Node Feature Augmentation on Graphs.', 'Semi-Supervised Learning for Time Series Collected at a Low Sampling Rate.', 'Understanding Inter-Session Intentions via Complex Logical Reasoning.', 'Towards Robust Information Extraction via Binomial Distribution Guided Counterpart Sequence.', 'Meta Clustering of Neural Bandits.', 'Improved Active Covering via Density-Based Space Transformation.', 'Graph Mamba: Towards Learning on Graphs with State Space Models.', 'Evading Community Detection via Counterfactual Neighborhood Search.', 'FaultInsight: Interpreting Hyperscale Data

In [6]:
print(len(titles), len(authors))


601 601


In [7]:
def graph_mask(title):
    t = title.lower()
    conditions = ['graph neural networks', 'gnns', 'gnn', 'graph', 'graph learning', 'graph embedding', 'network embedding']
    return np.any([c in t for c in conditions])

In [8]:
with open(f'2024-KDD.txt', 'w', encoding='utf8') as fw:
    for t, n in zip(titles, authors):
        if graph_mask(t):
            fw.write(f'1. **{t}**\n\n')
            fw.write(f'    *{n}*\n\n')